# Notes and Snippets for "Advent of Code"
<https://adventofcode.com/>

Mostly for my own sake, to collect and remember trick of general use. 

## Some very use full people and links 

### People
* HyperNeutrino: detailed walk-through of problems, [YouTube](https://www.youtube.com/@hyper-neutrino), [GitHub](https://github.com/hyperneutrino/advent-of-code) 


### Other links
Reddit post with list of Python snippets from AoC  
<https://www.reddit.com/r/adventofcode/comments/1gsl4fm/share_your_favorite_tricks_and_snippets/>

Reddit post, will all old AoC problems  
<https://www.reddit.com/r/adventofcode/comments/1gdw4cj/450_stars_a_categorization_and_megaguide/>


## General Snippets

### packages

```Python

import copy # needed for deepcopy

from collections import defaultdict #Allows for not checking if a key exists. 
# types =  defaultdict(list) # here we can e.g. append to any key as default will be an empty list

from collections import Counter
# s = "aabbbcdd"
# Get counts of each character
#counts = Counter(s)
#print(counts)  # Counter({'b': 3, 'd': 2, 'a': 2, 'c': 1})

from collections import deque
# see also https://docs.python.org/3/library/collections.html


from functools import cache 
# Add @cache before, e.g., recursive functions, for very efficient caching. 
# Requires that the function is called with Immutable options.

import itertools # for permutations, see https://docs.python.org/3/library/itertools.html

import networkx as nx # for graphs, a large library

import heapq  # for priority queue in e.g.  Dijkstra's algorithm
    
```

## 2023 
Some solutions from 2023 where I did not get through that much, and some from 2025 training.

<https://adventofcode.com/2023>

All solutions can be found on:  
<https://github.com/BoJakobsen/AoC2023>

### Day 7: Camel Cards
<https://adventofcode.com/2023/day/7>

**Poker like game.** 

Solved using counting and **costume sorting** using key=. 
Remember key = for sort as e.g. in this (from Claude.ai)

```Python
# Define custom card strength
card_order = {'A': 14, 'K': 13, 'Q': 12, 'J': 11, 'T': 10, 
              '9': 9, '8': 8, '7': 7, '6': 6, '5': 5, 
              '4': 4, '3': 3, '2': 2}

def hand_strength(hand_str):
    """Convert hand string to tuple of card values for comparison"""
    return tuple(card_order[card] for card in hand_str)

sorted_plays = sorted(plays, key=lambda x: hand_strength(x[0]))
```


**Part 2**: Very nice use of itertools for selecting index's to a list for increasing/chaning (distributing jokers here)
 values in a systematic way. 

```Python
cnt = list(cnter.values())
for combo in combinations_with_replacement(range(len(cnt)), Js):
    cnt_new = cnt.copy()
    for kk in combo:
        cnt_new[kk] += 1
````

`combinations_with_replacement(range(len(cnt)), Js)`
Generates all sets of length Js from the set, can in this case be used to access all combinations of index's




### Day 14: Parabolic Reflector Dish
<https://adventofcode.com/2023/day/14>

String manipulation, moving round rocks "O" up to square rocks "#" not moving:  
`O.#..O.#.#`

Nice to remember: 
* Flip list of list in diagonal (exchange rows and columns)
```Python
lines_f = list(zip(*lines))
```
* Rotation of lists of lists
```Python
# Clockwise rotation
def cw_rot(lines_f):
    return [list(row) for row in zip(*lines_f[::-1])]

# Counter clockwise rotation
def ccw_rot(lines_f):  # counter clockwise
    return list(zip(*lines_f))[::-1]
```
* Convert list of list to tuple of tuples (non-mutable)
```Python
def matrix_to_key(matrix):
    """Convert matrix to hashable tuple"""
    return tuple(tuple(row) for row in matrix)
```
* Nice trick from HyperNeutrino, using a split and sort on the strings for moving the stones. 

### Day 17: Clumsy Crucible

Dijkstra search algorithm, with non-standard definition of connected states. 
Important to realize that the state is defined both by position and move direction and number of times moved in that direction. 

Rather clean implementation.


## 2024

First year where I did it all (but needed some inspiration for some of the problems)

All solutions can be found on:  
<https://github.com/BoJakobsen/AoC2024>

###  Day 16: Reindeer Maze

Maze solving using Dijkstra's algorithm

Two versions the latter inspired by HuperNeutrinos solution 

## Algorithms 

### Dijkstra search algorithm

Nice example from Claude.ai 

* For general graphs
```Python
import heapq

def dijkstra(graph, start, end):
    """
    Find shortest path from start to end using Dijkstra's algorithm.
    
    Args:
        graph: dict of dict, graph[node][neighbor] = weight
        start: starting node
        end: ending node
    
    Returns:
        (distance, path) tuple
    """
    # Priority queue: (distance, node)
    pq = [(0, start)]
    
    # Track shortest distance to each node
    distances = {start: 0}
    
    # Track previous node in optimal path
    previous = {start: None}
    
    # Track visited nodes
    visited = set()
    
    while pq:
        current_dist, current = heapq.heappop(pq)
        
        # Skip if already visited
        if current in visited:
            continue
        
        visited.add(current)
        
        # Found the destination
        if current == end:
            break
        
        # Check all neighbors
        for neighbor, weight in graph[current].items():
            if neighbor in visited:
                continue
            
            # Calculate new distance
            new_dist = current_dist + weight
            
            # Update if we found a shorter path
            if neighbor not in distances or new_dist < distances[neighbor]:
                distances[neighbor] = new_dist
                previous[neighbor] = current
                heapq.heappush(pq, (new_dist, neighbor))
    
    # Reconstruct path
    path = []
    current = end
    while current is not None:
        path.append(current)
        current = previous.get(current)
    path.reverse()
    
    return distances.get(end, float('inf')), path


# Example usage:
graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

distance, path = dijkstra(graph, 'A', 'D')
print(f"Shortest distance: {distance}")  # 4
print(f"Path: {' -> '.join(path)}")      # A -> B -> C -> D
```
* For Maps
```Python
import heapq

def dijkstra_grid(grid):
    """
    Find shortest path in a 2D grid from top-left to bottom-right.
    Grid values represent the cost to enter that cell.
    """
    rows, cols = len(grid), len(grid[0])
    start = (0, 0)
    end = (rows - 1, cols - 1)
    
    # Priority queue: (distance, row, col)
    pq = [(0, 0, 0)]
    
    # Track shortest distance to each position
    distances = {start: 0}
    
    # Visited set
    visited = set()
    
    # Directions: up, right, down, left
    directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    
    while pq:
        dist, r, c = heapq.heappop(pq)
        
        # Skip if already visited
        if (r, c) in visited:
            continue
        
        visited.add((r, c))
        
        # Found destination
        if (r, c) == end:
            return dist
        
        # Check all 4 neighbors
        for dr, dc in directions:
            nr, nc = r + dr, c + dc
            
            # Check bounds
            if 0 <= nr < rows and 0 <= nc < cols:
                if (nr, nc) in visited:
                    continue
                
                # Cost is current distance + cost to enter new cell
                new_dist = dist + grid[nr][nc]
                
                # Update if shorter path found
                if (nr, nc) not in distances or new_dist < distances[(nr, nc)]:
                    distances[(nr, nc)] = new_dist
                    heapq.heappush(pq, (new_dist, nr, nc))
    
    return float('inf')  # No path found


# Example usage:
grid = [
    [1, 3, 1],
    [1, 5, 1],
    [4, 2, 1]
]

result = dijkstra_grid(grid)
print(f"Minimum cost: {result}")  # Output: 7 (path: 1→1→1→2→1)
```
